<a href="https://colab.research.google.com/github/sumitmsc/PythonPractice/blob/main/SingleAgentMultiTask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#!/usr/bin/env python3
"""
Single-Agent, Multi-Task Demo (Planning • Strategy • Autonomy)
- One agent processes a queue of natural-language tasks/goals sequentially.
- Each task is handled end-to-end (plan → strategy → schedule → forecast → brief).
- Shows: single decision-maker, multiple tasks, lightweight tool-use, and memory per task.

Run examples:
    python single_agent_multi_task.py \
        --goal "Plan a 3-day social promo for a sports brand with ₹50,000 budget to boost ecommerce sales." \
        --goal "Create a 2-day influencer sprint for launch with ₹20,000 budget." \
        --brand FleetAthlete --theme Speed

    # Or load from a JSON file containing a list of goal strings:
    python single_agent_multi_task.py --goals_file goals.json

Outputs:
- Console summary for each task
- A combined report saved as multi_task_report.txt
"""
from __future__ import annotations
import argparse
import json
import math
import random
import re
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional

# -----------------------------
# "Tools" the agent can use
# -----------------------------

class CalculatorTool:
    """Tiny safe-ish calculator for simple expressions like '50000*0.25'."""
    ALLOWED = set("0123456789.+-*/()% ")

    def run(self, expr: str) -> float:
        if not set(expr) <= self.ALLOWED:
            raise ValueError("Unsafe expression.")
        return float(eval(expr, {"__builtins__": None}, {}))

class KnowledgeBaseTool:
    """Mocked 'research' tool with pragmatic defaults the agent can consult."""
    KB = {
        "cpc_india_sports_paid_social": 6.5,       # ₹ per click (rough heuristic)
        "ctr_organic_social": 0.015,               # 1.5% click-through
        "conv_rate_ecom": 0.02,                    # 2% site conversion
        "avg_order_value": 1800.0,                 # ₹
        "influencer_micro_cost_per_post": 5000.0,  # ₹
        "email_ctr": 0.03,
        "email_conv": 0.02,
        "best_posting_times": ["7:30 AM", "12:30 PM", "7:30 PM"],
        "channels": ["Paid Social", "Organic Social", "Influencers", "Email/SMS", "Content/Blog"],
    }

    def lookup(self, key: str) -> Any:
        return self.KB.get(key)

class NoteTool:
    """Simple in-run memory (resets each run_many call)."""
    def __init__(self):
        self.memory: Dict[str, Any] = {}

    def set(self, k: str, v: Any):
        self.memory[k] = v

    def get(self, k: str, default=None):
        return self.memory.get(k, default)

    def add_list(self, k: str, v: Any):
        self.memory.setdefault(k, []).append(v)

class SchedulerTool:
    def make_schedule(self, days: int, slots: Optional[List[str]] = None) -> List[Dict[str, Any]]:
        if slots is None:
            slots = ["Morning", "Afternoon", "Evening"]
        plan = []
        for d in range(1, max(1, days) + 1):
            plan.append({
                "day": d,
                "slots": {s: [] for s in slots}
            })
        return plan

class IdeaTool:
    def brainstorm_posts(self, brand: str, theme: str, n: int = 5) -> List[str]:
        random.seed(42 + len(brand) + len(theme))
        verbs = ["Crush", "Chase", "Own", "Level-Up", "Ignite", "Unlock"]
        angles = ["form tips", "micro-workouts", "gear care", "coach wisdom", "community stories", "before/after"]
        ideas = []
        for _ in range(n * 2):  # extra draws to increase variety
            v = random.choice(verbs)
            a = random.choice(angles)
            ideas.append(f"{v} your game: {a} ({theme} × {brand})")
        # de-dup while preserving order
        seen, out = set(), []
        for i in ideas:
            if i not in seen:
                seen.add(i)
                out.append(i)
            if len(out) == n:
                break
        return out

# -----------------------------
# Planner & Strategy Modules
# -----------------------------

@dataclass
class Step:
    name: str
    done: bool = False
    details: Dict[str, Any] = field(default_factory=dict)

class Planner:
    BUDGET_REGEX = r"(₹|INR|\$)?\s?([0-9][0-9,\.]*)"
    DAYS_REGEX = r"(\d+)\s*(day|days|d)\b"

    def parse_budget(self, text: str) -> Optional[float]:
        matches = re.findall(self.BUDGET_REGEX, text, flags=re.I)
        nums = []
        for _, raw in matches:
            try:
                nums.append(float(raw.replace(",", "")))
            except:
                pass
        return max(nums) if nums else None

    def parse_days(self, text: str) -> Optional[int]:
        m = re.search(self.DAYS_REGEX, text, flags=re.I)
        return int(m.group(1)) if m else None

    def plan(self, goal: str) -> Dict[str, Any]:
        budget = self.parse_budget(goal)
        days = self.parse_days(goal)
        steps = [
            Step("Clarify constraints", details={"need": ["budget", "days"]}),
            Step("Choose channels and split budget"),
            Step("Create content ideas"),
            Step("Build schedule & checklist"),
            Step("Forecast outcomes"),
            Step("Assemble final brief"),
        ]
        return {"budget": budget, "days": days, "steps": steps}

class Strategist:
    def __init__(self, kb: KnowledgeBaseTool, calc: CalculatorTool):
        self.kb = kb
        self.calc = calc

    def choose_channels(self, budget: float, days: int) -> Dict[str, Any]:
        if budget < 20000:
            alloc = {"Organic Social": 0.0, "Influencers": 0.6, "Content/Blog": 0.2, "Email/SMS": 0.2}
        elif budget < 100000:
            alloc = {"Paid Social": 0.5, "Influencers": 0.2, "Organic Social": 0.15, "Email/SMS": 0.15}
        else:
            alloc = {"Paid Social": 0.6, "Influencers": 0.2, "Content/Blog": 0.1, "Email/SMS": 0.1}
        split = {k: round(v, 2) for k, v in alloc.items()}
        rupees = {k: round(budget * v) for k, v in split.items()}
        return {"allocation": split, "rupees": rupees, "days": days}

    def simple_forecast(self, rupees: Dict[str, float]) -> Dict[str, Any]:
        cpc = self.kb.lookup("cpc_india_sports_paid_social")
        conv = self.kb.lookup("conv_rate_ecom")
        aov = self.kb.lookup("avg_order_value")
        influencer_cost = self.kb.lookup("influencer_micro_cost_per_post")

        results: Dict[str, Dict[str, Any]] = {}

        if "Paid Social" in rupees and rupees["Paid Social"] > 0 and cpc:
            clicks = rupees["Paid Social"] / cpc
            orders = clicks * conv
            rev = orders * aov
            results["Paid Social"] = {"clicks": int(clicks), "orders": int(orders), "revenue": int(rev)}

        if "Influencers" in rupees and rupees["Influencers"] > 0 and influencer_cost:
            posts = max(1, int(rupees["Influencers"] // influencer_cost))
            clicks = posts * 150
            orders = clicks * conv
            rev = orders * aov
            results["Influencers"] = {"posts": posts, "clicks": int(clicks), "orders": int(orders), "revenue": int(rev)}

        if "Organic Social" in rupees:
            clicks = 400
            orders = clicks * conv
            results["Organic Social"] = {"clicks": clicks, "orders": int(orders)}

        if "Email/SMS" in rupees:
            list_size = 10000
            ctr = self.kb.lookup("email_ctr")
            econv = self.kb.lookup("email_conv")
            clicks = list_size * ctr
            orders = clicks * econv
            rev = orders * aov
            results["Email/SMS"] = {"clicks": int(clicks), "orders": int(orders), "revenue": int(rev)}

        totals = {
            "orders": sum(v.get("orders", 0) for v in results.values()),
            "revenue": sum(v.get("revenue", 0) for v in results.values())
        }
        results["TOTALS"] = totals
        return results

# -----------------------------
# Tasks & Agent
# -----------------------------

@dataclass
class Task:
    id: int
    goal: str
    status: str = "queued"  # queued → running → done/failed
    result: Optional["AgentOutput"] = None
    error: Optional[str] = None

@dataclass
class AgentOutput:
    goal: str
    constraints: Dict[str, Any]
    strategy: Dict[str, Any]
    schedule: List[Dict[str, Any]]
    ideas: List[str]
    forecast: Dict[str, Any]
    brief: str

class SingleAgent:
    def __init__(self):
        self.calc = CalculatorTool()
        self.kb = KnowledgeBaseTool()
        self.scheduler = SchedulerTool()
        self.ideas = IdeaTool()
        self.planner = Planner()
        self.strategist = Strategist(self.kb, self.calc)
        # fresh NoteTool per run_many() call

    # -------- single-task run --------
    def run_one(self, goal: str, brand: str = "YourSportsBrand", theme: str = "Performance") -> AgentOutput:
        notes = NoteTool()
        plan = self.planner.plan(goal)
        budget, days = plan["budget"], plan["days"]

        if budget is None:
            budget = 50000.0
        if days is None:
            days = 3

        notes.set("constraints", {"budget": budget, "days": days})

        strat = self.strategist.choose_channels(budget, days)
        notes.set("strategy", strat)

        content_ideas = self.ideas.brainstorm_posts(brand, theme, n=6)
        notes.set("ideas", content_ideas)

        sched = self.scheduler.make_schedule(days, slots=self.kb.lookup("best_posting_times"))
        for day in sched:
            for t in day["slots"]:
                day["slots"][t] = [
                    f"Post: {random.choice(content_ideas)}",
                    "Story/Reel: behind-the-scenes",
                ]
        notes.set("schedule", sched)

        forecast = self.strategist.simple_forecast(strat["rupees"])
        notes.set("forecast", forecast)

        brief = self._assemble_brief(goal, brand, theme, notes.get("constraints"), strat, sched, content_ideas, forecast)

        return AgentOutput(
            goal=goal,
            constraints=notes.get("constraints"),
            strategy=strat,
            schedule=sched,
            ideas=content_ideas,
            forecast=forecast,
            brief=brief,
        )

    def _assemble_brief(self, goal, brand, theme, constraints, strat, sched, ideas, forecast) -> str:
        lines: List[str] = []
        lines.append(f"# Launch Brief\n")
        lines.append(f"**Goal:** {goal}")
        lines.append(f"**Brand/Theme:** {brand} / {theme}")
        lines.append(f"**Constraints:** Budget ₹{int(constraints['budget'])}, Duration {constraints['days']} days\n")

        lines.append("## Channel Strategy & Budget Split")
        for k, v in strat["allocation"].items():
            lines.append(f"- {k}: {int(v*100)}% (₹{strat['rupees'][k]})")

        lines.append("\n## Content Ideas (shortlist)")
        for i in ideas:
            lines.append(f"- {i}")

        lines.append("\n## Day-by-Day Schedule")
        for day in sched:
            lines.append(f"- **Day {day['day']}**")
            for slot, tasks in day["slots"].items():
                for task in tasks:
                    lines.append(f"  - {slot}: {task}")

        lines.append("\n## Rough Forecast (very rough, for planning only)")
        for ch, vals in forecast.items():
            if ch == "TOTALS":
                continue
            pretty = ", ".join(f"{k}={v}" for k, v in vals.items())
            lines.append(f"- {ch}: {pretty}")
        totals = forecast.get("TOTALS", {})
        lines.append(f"- **TOTALS:** orders={totals.get('orders', 0)}, revenue=₹{totals.get('revenue', 0)}")

        lines.append("\n## Risks & Mitigations")
        lines.append("- Creative fatigue → rotate formats daily; reuse top winners only.")
        lines.append("- Tracking issues → verify UTM and pixels before Day 1.")
        lines.append("- Inventory runouts → sync with ops; set real scarcity only.")

        lines.append("\n## Success Metrics")
        lines.append("- CTR, CPC (paid), follower growth (organic), email CTR/conv, total orders & revenue.")

        return "\n".join(lines)

    # -------- multi-task orchestration --------
    def run_many(self, goals: List[str], brand: str = "YourSportsBrand", theme: str = "Performance") -> List[Task]:
        tasks: List[Task] = [Task(id=i+1, goal=g) for i, g in enumerate(goals)]
        for t in tasks:
            t.status = "running"
            try:
                result = self.run_one(t.goal, brand=brand, theme=theme)
                t.result = result
                t.status = "done"
            except Exception as e:
                t.status = "failed"
                t.error = str(e)
        return tasks

# -----------------------------
# CLI
# -----------------------------

# Removed argument parsing and main execution block for Colab compatibility

def run_agent_in_colab(goals: List[str], brand: str = "YourSportsBrand", theme: str = "Performance") -> List[Task]:
    """Helper function to run the SingleAgent in a Colab environment."""
    agent = SingleAgent()
    tasks = agent.run_many(goals, brand=brand, theme=theme)

    # Console summary
    for t in tasks:
        print(f"=== Task #{t.id} | {t.status.upper()} ===")
        print(t.goal)
        if t.status == "done" and t.result:
            totals = t.result.forecast.get("TOTALS", {})
            print("Constraints:", t.result.constraints)
            print("Totals:", totals)
        elif t.error:
            print("Error:", t.error)
        print()

    # Write combined report
    report_path = "multi_task_report.txt"
    write_report(tasks, path=report_path)
    print(f"Combined report written to: {report_path}")

    return tasks


def write_report(tasks: List[Task], path: str = "multi_task_report.txt") -> str:
    lines: List[str] = []
    lines.append("=== SINGLE AGENT • MULTI TASK REPORT ===\n")
    for t in tasks:
        lines.append(f"Task #{t.id}: {t.goal}")
        lines.append(f"Status: {t.status}\n")
        if t.status == "done" and t.result:
            r = t.result
            lines.append(f"Constraints: {r.constraints}")
            lines.append(f"Strategy: {json.dumps(r.strategy, indent=2)}")
            lines.append(f"Top Ideas: {r.ideas[:3]}")
            lines.append(f"Forecast Totals: {r.forecast.get('TOTALS', {})}\n")
            lines.append(r.brief)
            lines.append("\n" + ("-" * 60) + "\n")
        elif t.error:
            lines.append(f"Error: {t.error}\n")
    with open(path, "w", encoding="utf-8") as f:
        f.write("\n".join(lines))
    return path
'''
    goals = [
    "Plan a 3-day social promo for a sports brand with ₹50,000 budget to boost ecommerce sales.",
    "Create a 2-day influencer sprint for launch with ₹20,000 budget."
]
brand = "FleetAthlete"
theme = "Speed"

run_agent_in_colab(goals=goals, brand=brand, theme=theme)'''

NameError: name 'goals' is not defined

In [3]:
#!/usr/bin/env python3
"""
Multi‑Agent • Multi‑Task Demo (Planning • Strategy • Autonomy with Roles + Queue)

- Processes a queue of natural‑language goals (tasks) with multiple specialized agents.
- Each task runs through the same pipeline: Plan → Research → Strategy → Content → Schedule → Forecast → Critique → Re‑Forecast → Write
- Outputs per‑task briefs + a combined report and a global trace log.

Run examples:
  python multi_agent_multi_task.py \
    --goal "Plan a 3-day social promo for a sports brand with ₹50,000 budget to boost ecommerce sales." \
    --goal "Create a 2-day influencer sprint for launch with ₹20,000 budget." \
    --brand FleetAthlete --theme Speed

  # Or from a JSON file:
  python multi_agent_multi_task.py --goals_file goals.json --brand FleetAthlete --theme Speed

Artifacts:
- Per task: brief_<taskid>.txt
- Combined: combined_report.txt
- Global trace: trace_all.json
"""
from __future__ import annotations
import argparse
import json
import random
import re
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional

# -----------------------------
# Shared tools & types
# -----------------------------

class CalculatorTool:
    ALLOWED = set("0123456789.+-*/()% ")
    def run(self, expr: str) -> float:
        if not set(expr) <= self.ALLOWED:
            raise ValueError("Unsafe expression.")
        return float(eval(expr, {"__builtins__": None}, {}))

class KnowledgeBaseTool:
    KB = {
        "cpc_india_sports_paid_social": 6.5,
        "ctr_organic_social": 0.015,
        "conv_rate_ecom": 0.02,
        "avg_order_value": 1800.0,
        "influencer_micro_cost_per_post": 5000.0,
        "email_ctr": 0.03,
        "email_conv": 0.02,
        "best_posting_times": ["7:30 AM", "12:30 PM", "7:30 PM"],
        "channels": ["Paid Social", "Organic Social", "Influencers", "Email/SMS", "Content/Blog"],
    }
    def lookup(self, key: str) -> Any:
        return self.KB.get(key)

@dataclass
class Blackboard:
    task_id: int
    goal: str
    brand: str
    theme: str
    data: Dict[str, Any] = field(default_factory=dict)
    notes: List[Dict[str, Any]] = field(default_factory=list)
    status: str = "queued"  # queued → running → done/failed
    error: Optional[str] = None
    def set(self, k: str, v: Any):
        self.data[k] = v
    def get(self, k: str, default=None):
        return self.data.get(k, default)
    def log(self, agent: str, message: str, payload: Optional[Dict[str, Any]] = None):
        self.notes.append({"task": self.task_id, "agent": agent, "message": message, "payload": payload or {}})

# -----------------------------
# Agents
# -----------------------------

class PlanningAgent:
    NAME = "PlanningAgent"
    BUDGET_REGEX = r"(₹|INR|\$)?\s?([0-9][0-9,\.]*)"
    DAYS_REGEX = r"(\d+)\s*(day|days|d)\b"

    def run(self, bb: Blackboard):
        text = bb.goal
        # parse budget (largest number) and days (first match)
        matches = re.findall(self.BUDGET_REGEX, text, flags=re.I)
        nums = []
        for _, raw in matches:
            try:
                nums.append(float(raw.replace(",", "")))
            except:  # noqa: E722
                pass
        budget = max(nums) if nums else 50000.0
        m = re.search(self.DAYS_REGEX, text, flags=re.I)
        days = int(m.group(1)) if m else 3
        steps = [
            "Clarify constraints",
            "Research heuristics",
            "Choose channels & budget split",
            "Generate content ideas",
            "Build schedule",
            "Forecast outcomes",
            "Critique & improve",
            "Assemble final brief",
        ]
        bb.set("constraints", {"budget": budget, "days": days})
        bb.set("steps", steps)
        bb.log(self.NAME, "Parsed constraints and planned steps", {"budget": budget, "days": days, "steps": steps})

class ResearchAgent:
    NAME = "ResearchAgent"
    def __init__(self, kb: KnowledgeBaseTool):
        self.kb = kb
    def run(self, bb: Blackboard):
        heur = {
            "cpc": self.kb.lookup("cpc_india_sports_paid_social"),
            "conv": self.kb.lookup("conv_rate_ecom"),
            "aov": self.kb.lookup("avg_order_value"),
            "influencer_cost": self.kb.lookup("influencer_micro_cost_per_post"),
            "email_ctr": self.kb.lookup("email_ctr"),
            "email_conv": self.kb.lookup("email_conv"),
            "best_times": self.kb.lookup("best_posting_times"),
        }
        bb.set("heuristics", heur)
        bb.log(self.NAME, "Loaded planning heuristics", heur)

class StrategyAgent:
    NAME = "StrategyAgent"
    def run(self, bb: Blackboard):
        budget = bb.get("constraints")["budget"]
        days = bb.get("constraints")["days"]
        if budget < 20000:
            alloc = {"Organic Social": 0.0, "Influencers": 0.6, "Content/Blog": 0.2, "Email/SMS": 0.2}
        elif budget < 100000:
            alloc = {"Paid Social": 0.5, "Influencers": 0.2, "Organic Social": 0.15, "Email/SMS": 0.15}
        else:
            alloc = {"Paid Social": 0.6, "Influencers": 0.2, "Content/Blog": 0.1, "Email/SMS": 0.1}
        split = {k: round(v, 2) for k, v in alloc.items()}
        rupees = {k: round(budget * v) for k, v in split.items()}
        strat = {"allocation": split, "rupees": rupees, "days": days}
        bb.set("strategy", strat)
        bb.log(self.NAME, "Selected channels and budget split", strat)

class ContentAgent:
    NAME = "ContentAgent"
    def brainstorm(self, brand: str, theme: str, n: int = 6) -> List[str]:
        random.seed(42 + len(brand) + len(theme))
        verbs = ["Crush", "Chase", "Own", "Level-Up", "Ignite", "Unlock"]
        angles = ["form tips", "micro-workouts", "gear care", "coach wisdom", "community stories", "before/after"]
        ideas = []
        for _ in range(n * 2):
            v = random.choice(verbs)
            a = random.choice(angles)
            ideas.append(f"{v} your game: {a} ({theme} × {brand})")
        seen, out = set(), []
        for i in ideas:
            if i not in seen:
                seen.add(i)
                out.append(i)
            if len(out) == n:
                break
        return out
    def run(self, bb: Blackboard):
        ideas = self.brainstorm(bb.brand, bb.theme)
        bb.set("ideas", ideas)
        bb.log(self.NAME, "Generated content ideas", {"count": len(ideas)})

class SchedulingAgent:
    NAME = "SchedulingAgent"
    def run(self, bb: Blackboard):
        days = bb.get("constraints")["days"]
        times = (bb.get("heuristics") or {}).get("best_times") or ["Morning", "Afternoon", "Evening"]
        ideas = bb.get("ideas", [])
        schedule: List[Dict[str, Any]] = []
        for d in range(1, max(1, days) + 1):
            slots = {}
            for t in times:
                slots[t] = [
                    f"Post: {random.choice(ideas) if ideas else 'TBD'}",
                    "Story/Reel: behind-the-scenes",
                ]
            schedule.append({"day": d, "slots": slots})
        bb.set("schedule", schedule)
        bb.log(self.NAME, "Built day-by-day schedule", {"days": days, "slots": len(times)})

class ForecastAgent:
    NAME = "ForecastAgent"
    def __init__(self, kb: KnowledgeBaseTool):
        self.kb = kb
    def run(self, bb: Blackboard):
        strat = bb.get("strategy")
        if not strat:
            bb.log(self.NAME, "No strategy available; skipping forecast", {})
            return
        rupees = strat["rupees"]
        conv = self.kb.lookup("conv_rate_ecom")
        aov = self.kb.lookup("avg_order_value")
        cpc = self.kb.lookup("cpc_india_sports_paid_social")
        infl_cost = self.kb.lookup("influencer_micro_cost_per_post")
        email_ctr = self.kb.lookup("email_ctr")
        email_conv = self.kb.lookup("email_conv")

        results: Dict[str, Dict[str, Any]] = {}
        if rupees.get("Paid Social", 0) > 0 and cpc:
            clicks = rupees["Paid Social"] / cpc
            orders = clicks * conv
            revenue = orders * aov
            results["Paid Social"] = {"clicks": int(clicks), "orders": int(orders), "revenue": int(revenue)}
        if rupees.get("Influencers", 0) > 0 and infl_cost:
            posts = max(1, int(rupees["Influencers"] // infl_cost))
            clicks = posts * 150
            orders = clicks * conv
            revenue = orders * aov
            results["Influencers"] = {"posts": posts, "clicks": int(clicks), "orders": int(orders), "revenue": int(revenue)}
        if "Organic Social" in rupees:
            clicks = 400
            orders = clicks * conv
            results["Organic Social"] = {"clicks": clicks, "orders": int(orders)}
        if "Email/SMS" in rupees:
            list_size = 10000
            clicks = list_size * email_ctr
            orders = clicks * email_conv
            revenue = orders * aov
            results["Email/SMS"] = {"clicks": int(clicks), "orders": int(orders), "revenue": int(revenue)}
        totals = {"orders": sum(v.get("orders", 0) for v in results.values()),
                  "revenue": sum(v.get("revenue", 0) for v in results.values())}
        results["TOTALS"] = totals
        bb.set("forecast", results)
        bb.log(self.NAME, "Created rough forecast", totals)

class CriticAgent:
    NAME = "CriticAgent"
    def run(self, bb: Blackboard):
        constraints = bb.get("constraints")
        strat = bb.get("strategy")
        schedule = bb.get("schedule")
        if not (constraints and strat):
            bb.log(self.NAME, "Missing inputs; skipping critique", {})
            return
        issues: List[str] = []
        if constraints["budget"] < 20000 and strat["allocation"].get("Paid Social", 0) > 0:
            issues.append("Budget too low for Paid Social; shift to Influencers/Email/Organic.")
        if not schedule or not any(day["slots"] for day in schedule):
            issues.append("Schedule is empty.")
        if issues:
            alloc = strat["allocation"].copy()
            if any("Budget too low for Paid Social" in i for i in issues):
                spend = strat["rupees"].get("Paid Social", 0)
                alloc.pop("Paid Social", None)
                # redistribute proportionally to remaining or specified buckets
                targets = ["Influencers", "Email/SMS"]
                existing_sum = sum(alloc.values())
                remaining = 1.0 - existing_sum
                for i, k in enumerate(targets):
                    alloc[k] = round(alloc.get(k, 0.0) + (remaining * (0.6 if i == 0 else 0.4)), 2)
                rupees = {k: round(constraints["budget"] * v) for k, v in alloc.items()}
                strat = {"allocation": alloc, "rupees": rupees, "days": constraints["days"]}
                bb.set("strategy", strat)
        bb.log(self.NAME, "Critique complete", {"issues": issues})

class WriterAgent:
    NAME = "WriterAgent"
    def run(self, bb: Blackboard):
        c = bb.get("constraints") or {"budget": 0, "days": 0}
        strat = bb.get("strategy") or {"allocation": {}, "rupees": {}}
        ideas = bb.get("ideas", [])
        schedule = bb.get("schedule", [])
        forecast = bb.get("forecast", {})
        lines: List[str] = []
        lines.append(f"# Launch Brief — Task {bb.task_id}\n")
        lines.append(f"**Goal:** {bb.goal}")
        lines.append(f"**Brand/Theme:** {bb.brand} / {bb.theme}")
        lines.append(f"**Constraints:** Budget ₹{int(c['budget'])}, Duration {c['days']} days\n")
        lines.append("## Channel Strategy & Budget Split")
        for k, v in strat["allocation"].items():
            lines.append(f"- {k}: {int(v*100)}% (₹{strat['rupees'][k]})")
        lines.append("\n## Content Ideas (shortlist)")
        for i in ideas:
            lines.append(f"- {i}")
        lines.append("\n## Day-by-Day Schedule")
        for day in schedule:
            lines.append(f"- **Day {day['day']}**")
            for slot, tasks in day["slots"].items():
                for task in tasks:
                    lines.append(f"  - {slot}: {task}")
        lines.append("\n## Rough Forecast (very rough, for planning only)")
        for ch, vals in forecast.items():
            if ch == "TOTALS":
                continue
            pretty = ", ".join(f"{k}={v}" for k, v in vals.items())
            lines.append(f"- {ch}: {pretty}")
        totals = forecast.get("TOTALS", {})
        lines.append(f"- **TOTALS:** orders={totals.get('orders', 0)}, revenue=₹{totals.get('revenue', 0)}")
        lines.append("\n## Risks & Mitigations")
        lines.append("- Creative fatigue → rotate formats daily; reuse top winners only.")
        lines.append("- Tracking issues → verify UTM and pixels before Day 1.")
        lines.append("- Inventory runouts → sync with ops; set real scarcity only.")
        lines.append("\n## Success Metrics")
        lines.append("- CTR, CPC (paid), follower growth (organic), email CTR/conv, total orders & revenue.")
        brief = "\n".join(lines)
        bb.set("brief", brief)
        bb.log(self.NAME, "Assembled final brief", {"length": len(brief)})


# -----------------------------
# Orchestrator (multi-task)
# -----------------------------

class Orchestrator:
    def __init__(self):
        self.kb = KnowledgeBaseTool()
        self.pipeline = [
            PlanningAgent(),
            ResearchAgent(self.kb),
            StrategyAgent(),
            ContentAgent(),
            SchedulingAgent(),
            ForecastAgent(self.kb),
            CriticAgent(),
            ForecastAgent(self.kb),  # re-forecast if strategy changed
            WriterAgent(),
        ]

    def run_task(self, task_id: int, goal: str, brand: str, theme: str) -> Blackboard:
        bb = Blackboard(task_id=task_id, goal=goal, brand=brand, theme=theme)
        bb.status = "running"
        try:
            for agent in self.pipeline:
                agent.run(bb)
            bb.status = "done"
        except Exception as e:
            bb.status = "failed"
            bb.error = str(e)
            bb.log("Orchestrator", "Task failed", {"error": bb.error})
        return bb

    def run_many(self, goals: List[str], brand: str, theme: str) -> List[Blackboard]:
        results: List[Blackboard] = []
        for i, g in enumerate(goals, start=1):
            bb = self.run_task(task_id=i, goal=g, brand=brand, theme=theme)
            results.append(bb)
        return results


        combined_lines.append(f"Constraints: {bb.get('constraints')}")
